In [1]:
import requests
import re
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep

In [ ]:
variant_template = {
	'size': 'Medium',
	'units': 0,
	'waist_low': 10,
	'waist_high': 11,
}

updated_items = {
	'price': 0.00,
	'in_stock': False,
	'ml_per_unit_price': 0.00,
	'total_price': 0.00,
	'unit_price': 0.00,
}

In [2]:
driver = webdriver.Firefox()

In [20]:
urls = [
	'https://us.abuniverse.com/product/agz/',
	'https://us.abuniverse.com/product/bny4/',
	'https://us.abuniverse.com/product/cbc/',
	'https://us.abuniverse.com/product/cus/',
	'https://us.abuniverse.com/product/dno/',
	'https://us.abuniverse.com/product/lkg/',
	'https://us.abuniverse.com/product/pwz/',
	'https://us.abuniverse.com/product/pab/',
	'https://us.abuniverse.com/product/prp/',
	'https://us.abuniverse.com/product/prs/',
	'https://us.abuniverse.com/product/whi/',
	'https://us.abuniverse.com/product/siu/',
	'https://us.abuniverse.com/product/kdo/',
	'https://us.abuniverse.com/product/spc/',
	'https://us.abuniverse.com/product/sdk/',
]

In [34]:
with open('data.json') as file:
	stored_products = json.load(file)

In [33]:
rows = []
for url in urls:
	row = abu(url)
	rows.extend(row)

with open('sheet.json', 'w') as file:
	json.dump(rows, file, sort_keys=True, indent=4, separators=(',', ': '))

In [4]:
def get_response(url):
	with requests.get(url) as response:
		if response.status_code != 200:
			print(f'Error: {response.status_code} for {url}')
			return
		return response

def get_soup(url):
		return BeautifulSoup(get_page(url), 'html.parser')

def get_page(url):
	return get_response(url).text

def get_data(url):
	return get_response(url).json()

In [18]:
# ABU
def abu(url):
	rows = []
	soup = get_soup(url)
	stored_product = stored_products[url]
	base_items = stored_product['template']
	id = stored_product['id']
	for variation in get_data(f'https://us.abuniverse.com/wp-json/wc/store/products/{id}')['variations']:
		attributes = variation['attributes']
		if attributes[2]['value'] == 'scent' or attributes[0]['value'] == 'sample':
			continue
		variation_id = variation['id']
		data = get_data(f'https://us.abuniverse.com/wp-json/wc/store/products/{variation_id}')
		stored_size = stored_product['sizes'][attributes[1]['value']]
		size_label = stored_size['label']
		waist_low = stored_size['waist_low']
		waist_high = stored_size['waist_high']
		for units, ident in stored_product['units'].items():
			units = int(units)
			price_data = soup.find('label', {'for': ident}).string
			price = float(re.search('\d*\.\d*', price_data).group())
			unit_price = float(price / units)
			ml_per_unit_price = int(base_items['capacity'] / unit_price)
			total_price = price + base_items['shipping']
			in_stock = 'Yes' if data['is_in_stock'] else 'No'
			additional_items = {
				'size_label': size_label,
				'units': units,
				'waist_low': waist_low,
				'waist_high': waist_high,
				'price': price,
				'in_stock': in_stock,
				'ml_per_unit_price': ml_per_unit_price,
				'total_price': total_price,
				'unit_price': unit_price,
			}
			rows.append(base_items | additional_items)
	return rows

In [7]:
#Amazon
url = 'https://www.amazon.com/dp/B08GL65JKT'
rows = []

base_items = stored_products[url]['template']

driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

units = int(base_items['units'])

price = float(soup.find('span', {'class': 'apexPriceToPay'}).find('span', {'class': 'a-offscreen'}).string[1:])
in_stock = "Maybe"
unit_price = float(price / units)
ml_per_unit_price = int(base_items['capacity'] / unit_price)
total_price = float(price + base_items['shipping'])

additional_items = {
	'price': price,
	'in_stock': in_stock,
	'ml_per_unit_price': ml_per_unit_price,
	'total_price': total_price,
	'unit_price': unit_price,
}

rows.append(base_items | additional_items)

print(*rows, sep='\n')

{'backing': 'plastic', 'brand': 'ABDL Shop', 'capacity': 5000, 'name': 'Carousel', 'size_label': 'Large', 'notes': '', 'retailer': 'Amazon', 'shipping': 0.0, 'tapes': '2', 'units': 10, 'url': 'https://www.amazon.com/dp/B08GL65JKT', 'waist_high': 44, 'waist_low': 37, 'price': 39.99, 'in_stock': 'Maybe', 'ml_per_unit_price': 1250, 'total_price': 39.99, 'unit_price': 3.999}


In [8]:
#Bambino
url = 'https://bambinodiapers.com/products/new-cloudee-all-over-print-diapers.json'
rows = []
product = get_data(url)['product']
stored_product = stored_products[url]
base_items = stored_product['template']

for variant in product['variants']:
	if variant['option2'].startswith('1 Sample'):
		continue

	size = variant['option1'][:-10]
	stored_size = stored_product['sizes'][size]
	size_label = stored_size['label']
	waist_high = stored_size['waist_high']
	waist_low = stored_size['waist_low']
	units = int(variant['option2'][-2:])
	price = float(variant['price'])
	in_stock = 'Maybe'
	unit_price = float(price / units)
	ml_per_unit_price = int(base_items['capacity'] / unit_price)
	total_price = float(price + base_items['shipping'])

	additional_items = {
		'size_label': size_label,
		'units': units,
		'waist_low': waist_low,
		'waist_high': waist_high,
		'price': price,
		'in_stock': in_stock,
		'ml_per_unit_price': ml_per_unit_price,
		'total_price': total_price,
		'unit_price': unit_price,
	}

	rows.append(base_items | additional_items)

print(*rows, sep='\n')

{'backing': 'plastic', 'brand': 'Bambino', 'capacity': 5000, 'name': 'Cloudee', 'notes': '', 'retailer': 'Bambino', 'shipping': 0.0, 'tapes': '4', 'url': 'https://bambinodiapers.com/products/new-cloudee-all-over-print-diapers', 'size_label': 'Medium', 'units': 8, 'waist_low': 32, 'waist_high': 40, 'price': 30.99, 'in_stock': 'Maybe', 'ml_per_unit_price': 1290, 'total_price': 30.99, 'unit_price': 3.87375}
{'backing': 'plastic', 'brand': 'Bambino', 'capacity': 5000, 'name': 'Cloudee', 'notes': '', 'retailer': 'Bambino', 'shipping': 0.0, 'tapes': '4', 'url': 'https://bambinodiapers.com/products/new-cloudee-all-over-print-diapers', 'size_label': 'Medium', 'units': 48, 'waist_low': 32, 'waist_high': 40, 'price': 119.99, 'in_stock': 'Maybe', 'ml_per_unit_price': 2000, 'total_price': 119.99, 'unit_price': 2.4997916666666664}
{'backing': 'plastic', 'brand': 'Bambino', 'capacity': 5000, 'name': 'Cloudee', 'notes': '', 'retailer': 'Bambino', 'shipping': 0.0, 'tapes': '4', 'url': 'https://bambino

In [9]:
#InControl
url = 'https://incontroldiapers.com/incontrol-premium-nights-briefs-with-whiff-x-technology/'
driver.get(url)
rows = []
stored_product = stored_products[url]
base_items = stored_product['template']

buttons = dict(map(lambda button: (button.get_attribute('data-product-attribute-value'), button), driver.find_elements(By.CLASS_NAME, 'form-option')))

for units, units_id in stored_product['units'].items():
	units = int(units)
	buttons[units_id].click()
	for size in stored_product['sizes']:
		buttons[size['attribute']].click()

		capacity = size['capacity']
		size_label = size['label']
		waist_high = size['waist_high']
		waist_low = size['waist_low']
		price_element = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[1]/div/div[1]/section[2]/div/div[1]/div[3]/span[3]')
		price = float(re.search('\d+.\d+', price_element.text).group())
		in_stock = 'Maybe'
		unit_price = float(price / units)
		ml_per_unit_price = int(capacity / unit_price)
		total_price = float(price + base_items['shipping'])

		additional_items = {
			'capacity': capacity,
			'size_label': size_label,
			'units': units,
			'waist_low': waist_low,
			'waist_high': waist_high,
			'price': price,
			'in_stock': in_stock,
			'ml_per_unit_price': ml_per_unit_price,
			'total_price': total_price,
			'unit_price': unit_price,
		}

		rows.append(base_items | additional_items)
print(*rows, sep='\n')

{'backing': 'plastic', 'brand': 'InControl', 'name': 'Premium Nights', 'notes': '', 'retailer': 'InControl', 'shipping': 0.0, 'tapes': '4', 'url': 'https://incontroldiapers.com/incontrol-premium-nights-briefs-with-whiff-x-technology/', 'capacity': 5435, 'size_label': 'Large', 'units': 12, 'waist_low': 33, 'waist_high': 42, 'price': 10.99, 'in_stock': 'Maybe', 'ml_per_unit_price': 5934, 'total_price': 10.99, 'unit_price': 0.9158333333333334}
{'backing': 'plastic', 'brand': 'InControl', 'name': 'Premium Nights', 'notes': '', 'retailer': 'InControl', 'shipping': 0.0, 'tapes': '4', 'url': 'https://incontroldiapers.com/incontrol-premium-nights-briefs-with-whiff-x-technology/', 'capacity': 5217, 'size_label': 'Medium', 'units': 12, 'waist_low': 30, 'waist_high': 40, 'price': 31.99, 'in_stock': 'Maybe', 'ml_per_unit_price': 1956, 'total_price': 31.99, 'unit_price': 2.665833333333333}
{'backing': 'plastic', 'brand': 'InControl', 'name': 'Premium Nights', 'notes': '', 'retailer': 'InControl', '

In [10]:
#LittleForBig
url = 'https://www.littleforbig.com/wp-json/wc/store/products/155255'
rows = []
product = get_data(url)
stored_product = stored_products[url]
base_items = stored_product['template']

for variation in product['variations']:
	variant_id = variation['id']
	variant = variation['attributes'][0]['value']
	variant_data = get_data(f'https://www.littleforbig.com/wp-json/wc/store/products/{variant_id}')

	size = variant[:6]
	stored_size = stored_product['sizes'][size]
	size_label = stored_size['label']
	waist_low = int(stored_size['waist_low'])
	waist_high = int(stored_size['waist_high'])
	units = int(re.search('\d\d ', variant).group())
	price = float('{0}.{1}'.format(variant_data['prices']['price'][:-2], variant_data['prices']['price'][-2:]))
	in_stock = 'Yes' if data['is_in_stock'] else 'No'
	unit_price = float(price / units)
	ml_per_unit_price = int(base_items['capacity'] / unit_price)
	total_price = float(price + base_items['shipping'])

	additional_items = {
		'size_label': size_label,
		'units': units,
		'waist_low': waist_low,
		'waist_high': waist_high,
		'price': price,
		'in_stock': in_stock,
		'ml_per_unit_price': ml_per_unit_price,
		'total_price': total_price,
		'unit_price': unit_price,
	}
	
	rows.append(base_items | additional_items)

print(*rows, sep='\n')

155309
155310
155311
155312
155257
155258
{'backing': 'plastic', 'brand': 'LittleForBig', 'capacity': 5352, 'name': 'Blushing Baby', 'notes': 'assumed ISO', 'retailer': 'LittleForBig', 'shipping': 0.0, 'tapes': '4', 'url': 'https://www.littleforbig.com/product/blushing-baby-adult-diapers-10-pieces-packm-l/', 'size_label': 'Medium', 'units': 40, 'waist_low': 28, 'waist_high': 38, 'price': 139.99, 'in_stock': 'Yes', 'ml_per_unit_price': 1529, 'total_price': 139.99, 'unit_price': 3.49975}
{'backing': 'plastic', 'brand': 'LittleForBig', 'capacity': 5352, 'name': 'Blushing Baby', 'notes': 'assumed ISO', 'retailer': 'LittleForBig', 'shipping': 0.0, 'tapes': '4', 'url': 'https://www.littleforbig.com/product/blushing-baby-adult-diapers-10-pieces-packm-l/', 'size_label': 'Medium', 'units': 80, 'waist_low': 28, 'waist_high': 38, 'price': 248.99, 'in_stock': 'Yes', 'ml_per_unit_price': 1719, 'total_price': 248.99, 'unit_price': 3.112375}
{'backing': 'plastic', 'brand': 'LittleForBig', 'capacity':

In [11]:
#MyInnerbaby
url = 'https://myinnerbaby.com/products/ageplay-outfitters-seaside-princess-printed-adult-diaper.json'
rows = []
product = get_data(url)['product']
stored_product = stored_products[url]
base_items = stored_product['template']

for variant in product['variants']:
	if variant['option2'] != 'Bag of 10' and variant['option2'] != 'Case of 40':
		continue

	size = variant['option1']
	stored_size = stored_product['sizes'][size]
	size_label = stored_size['label']
	waist_low = int(stored_size['waist_low'])
	waist_high = int(stored_size['waist_high'])
	units = int(variant['option2'][-2:])
	price = float(variant['price'])
	in_stock = 'Maybe'
	unit_price = float(price / units)
	ml_per_unit_price = int(base_items['capacity'] / unit_price)
	total_price = float(price + base_items['shipping'])

	additional_items = {
		'size_label': size_label,
		'units': units,
		'waist_low': waist_low,
		'waist_high': waist_high,
		'price': price,
		'in_stock': in_stock,
		'ml_per_unit_price': ml_per_unit_price,
		'total_price': total_price,
		'unit_price': unit_price,
	}
	
	rows.append(base_items | additional_items)

print(*rows, sep='\n')

{'backing': 'plastic', 'brand': 'MyInnerBaby', 'capacity': 5000, 'name': 'Seaside Princess', 'notes': '', 'retailer': 'MyInnerBaby', 'shipping': 0.0, 'tapes': '4', 'url': 'https://myinnerbaby.com/products/ageplay-outfitters-seaside-princess-printed-adult-diaper', 'size_label': 'Medium', 'units': 10, 'waist_low': 32, 'waist_high': 40, 'price': 39.95, 'in_stock': 'Maybe', 'ml_per_unit_price': 1251, 'total_price': 39.95, 'unit_price': 3.995}
{'backing': 'plastic', 'brand': 'MyInnerBaby', 'capacity': 5000, 'name': 'Seaside Princess', 'notes': '', 'retailer': 'MyInnerBaby', 'shipping': 0.0, 'tapes': '4', 'url': 'https://myinnerbaby.com/products/ageplay-outfitters-seaside-princess-printed-adult-diaper', 'size_label': 'Medium', 'units': 40, 'waist_low': 32, 'waist_high': 40, 'price': 109.95, 'in_stock': 'Maybe', 'ml_per_unit_price': 1819, 'total_price': 109.95, 'unit_price': 2.7487500000000002}
{'backing': 'plastic', 'brand': 'MyInnerBaby', 'capacity': 5000, 'name': 'Seaside Princess', 'notes

In [12]:
#NorthShore
url = 'https://www.northshorecare.com/adult-diapers/adult-diapers-with-tabs/northshore-megamax-tab-style-briefs/northshore-megamax-tab-style-briefs-medium-case40-410s'
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')
rows = []
stored_product = stored_products[url]
base_items = stored_product['template']

price = float(soup.find('span', {'class', 'product-details__price_highlight'}).string[1:])
in_stock = 'Yes' if soup.find('span', {'class', 'icon icon-check'}) else 'No'
units = int(base_items['units'])
unit_price = price / units
ml_per_unit_price = int(base_items['capacity'] / unit_price)
total_price = float(price + base_items['shipping'])

additional_items = {
	'size_label': size_label,
		'price': price,
		'in_stock': in_stock,
		'ml_per_unit_price': ml_per_unit_price,
		'total_price': total_price,
		'unit_price': unit_price,
}

rows.append(base_items | additional_items)
print(*rows,sep='\n')

{'backing': 'plastic', 'brand': 'NorthShore', 'capacity': 6500, 'name': 'MEGAMAX', 'size_label': 'Large', 'notes': '', 'retailer': 'NorthShore', 'shipping': 0.0, 'tapes': '4', 'units': 40, 'url': 'https://www.northshorecare.com/adult-diapers/adult-diapers-with-tabs/northshore-megamax-tab-style-briefs/northshore-megamax-tab-style-briefs-medium-case40-410s', 'waist_high': 44, 'waist_low': 32, 'price': 114.99, 'in_stock': 'Yes', 'ml_per_unit_price': 2261, 'total_price': 114.99, 'unit_price': 2.8747499999999997}


In [15]:
#Rearz
url = 'https://rearz.ca/rearz-alpaca-overnight-diapers/'
driver.get(url)
sleep(4)
rows = []
stored_product = stored_products[url]
base_items = stored_product['template']



for units, units_id in stored_product['units'].items():
	units = int(units)
	driver.find_element(By.XPATH, units_id).click()
	for size in stored_product['sizes']:
		driver.find_element(By.XPATH, size['attribute']).click()
		sleep(4)
		size_label = size['label']
		waist_high = size['waist_high']
		waist_low = size['waist_low']
		price_element = driver.find_element(By.XPATH, '/html/body/div[3]/div[1]/div[1]/div/div[1]/div[2]/div[2]/div/div[3]/div[3]/span')
		price = float(re.search('\d+.\d+', price_element.text).group())
		in_stock = 'Maybe'
		unit_price = float(price / units)
		ml_per_unit_price = int(base_items['capacity'] / unit_price)
		total_price = float(price + base_items['shipping'])

		additional_items = {
			'size_label': size_label,
			'units': units,
			'waist_low': waist_low,
			'waist_high': waist_high,
			'price': price,
			'in_stock': in_stock,
			'ml_per_unit_price': ml_per_unit_price,
			'total_price': total_price,
			'unit_price': unit_price,
		}

		rows.append(base_items | additional_items)
		
print(*rows, sep='\n')

{'backing': 'plastic', 'brand': 'Rearz', 'capacity': 5800, 'name': 'Alpaca', 'notes': '', 'retailer': 'Rearz', 'shipping': 0.0, 'tapes': '4', 'url': 'https://rearz.ca/rearz-alpaca-overnight-diapers/', 'size_label': 'Large', 'units': 12, 'waist_low': 33, 'waist_high': 42, 'price': 45.99, 'in_stock': 'Maybe', 'ml_per_unit_price': 1513, 'total_price': 45.99, 'unit_price': 3.8325}
{'backing': 'plastic', 'brand': 'Rearz', 'capacity': 5800, 'name': 'Alpaca', 'notes': '', 'retailer': 'Rearz', 'shipping': 0.0, 'tapes': '4', 'url': 'https://rearz.ca/rearz-alpaca-overnight-diapers/', 'size_label': 'Medium', 'units': 12, 'waist_low': 30, 'waist_high': 40, 'price': 41.99, 'in_stock': 'Maybe', 'ml_per_unit_price': 1657, 'total_price': 41.99, 'unit_price': 3.499166666666667}
{'backing': 'plastic', 'brand': 'Rearz', 'capacity': 5800, 'name': 'Alpaca', 'notes': '', 'retailer': 'Rearz', 'shipping': 0.0, 'tapes': '4', 'url': 'https://rearz.ca/rearz-alpaca-overnight-diapers/', 'size_label': 'XLarge', 'un

In [16]:
# Tykables
url = 'https://tykables.com/products/str8up-pink.json'
rows = []
product = get_data(url)['product']
stored_product = stored_products[url]
base_items = stored_product['template']

for variant in product['variants']:
	if variant['option3'] != 'None' or variant['option2'] == '1 Sample':
		continue

	size = variant['option1']
	stored_size = stored_product['sizes'][size]
	size_label = stored_size['label']
	waist_low = int(stored_size['waist_low'])
	waist_high = int(stored_size['waist_high'])
	units = int(variant['option2'][:2])
	price = float(variant['price'])
	in_stock = 'Yes' if variant['inventory_quantity'] > 0 else 'No'
	unit_price = float(price / units)
	ml_per_unit_price = int(base_items['capacity'] / unit_price)
	total_price = float(price + base_items['shipping'])

	additional_items = {
		'size_label': size_label,
		'units': units,
		'waist_low': waist_low,
		'waist_high': waist_high,
		'price': price,
		'in_stock': in_stock,
		'ml_per_unit_price': ml_per_unit_price,
		'total_price': total_price,
		'unit_price': unit_price,
	}
	
	rows.append(base_items | additional_items)

print(*rows, sep='\n')

{'backing': 'plastic', 'brand': 'Tykables', 'capacity': 8000, 'name': 'Str8up Pink', 'notes': '', 'retailer': 'Tykables', 'shipping': 0.0, 'tapes': '4', 'url': 'https://tykables.com/products/str8up-pink', 'size_label': 'Medium', 'units': 10, 'waist_low': 28, 'waist_high': 36, 'price': 47.0, 'in_stock': 'Yes', 'ml_per_unit_price': 1702, 'total_price': 47.0, 'unit_price': 4.7}
{'backing': 'plastic', 'brand': 'Tykables', 'capacity': 8000, 'name': 'Str8up Pink', 'notes': '', 'retailer': 'Tykables', 'shipping': 0.0, 'tapes': '4', 'url': 'https://tykables.com/products/str8up-pink', 'size_label': 'Medium', 'units': 40, 'waist_low': 28, 'waist_high': 36, 'price': 125.0, 'in_stock': 'Yes', 'ml_per_unit_price': 2560, 'total_price': 125.0, 'unit_price': 3.125}
{'backing': 'plastic', 'brand': 'Tykables', 'capacity': 8000, 'name': 'Str8up Pink', 'notes': '', 'retailer': 'Tykables', 'shipping': 0.0, 'tapes': '4', 'url': 'https://tykables.com/products/str8up-pink', 'size_label': 'Medium', 'units': 80